# Notebook 3: Naive-Bayes-Klassifikation

<img src="https://balloon.tam.ch/share/b3ef6dd47539cc1a9d26b2bf8c778cdd" alt="Musik-Genres" style="float: right; width: 320px">

In diesem Jupyter-Notebook geht es darum, wie man mit Hilfe der Naive-Bayes-Klassifikation **Kategorien** voraussagt. 

In den Lernaufgaben unten erstellen wir aus den Lyrcis von Songs einer fiktiven Musiksammlung eine Genre-Klassifikator, der Songs basierend auf ihren Lyrcis einem Genre zuordnet.

Wir verwenden also die *abhängige* Variable `Genre` mit den folgenden Kategorien:
* Pop
* Soul
* Country
* Metal
* Hiphop

Als *unabhängige* Variablen verwenden wir die Häufigkeit der folgenden Wörter:
* love
* oh
* down
* got
* never
* feel
* let

## Definitionen

<div class="alert alert-block alert-info">
    
<strong>A-priori-Wahrscheinlichkeit $P(A)$</strong>

Unter der A-priori-Wahrscheinlichkeit $P(A)$ verstehen wir die aus *allgemeinem Vorwissen* geschätze Wahrscheinlichkeit eines Ereignisses $A$. Man zählt dafür die Anzahl Beobachtungen des Ereignisses $A$ (z.B. Anzahl Spam-Nachrichten) und dividiert sie durch die Anzahl aller Beobachtungen (Alle E-Mails in der Inbox).

**Beispiel**

Wir haben einen Datensatz mit 12 E-Mail-Nachrichten, wovon 8 normale Nachrichten (${\rm Ham}$) und 4 Spam-Nachrichten (${\rm Spam}$) sind. Die A-priori-Wahrscheinlichkeiten für die beiden Kategorien sind:

* $P({\rm Ham}) = \frac{8}{8+4} = \frac{2}{3}$
* $P({\rm Spam}) = \frac{4}{8+4} = \frac{1}{3}$

</div>

<div class="alert alert-block alert-info">

<strong>Bedingte Wahrscheinlichkeit $P(A | B)$</strong>

Wenn $A$ und $B$ beliebige Ereignisse sind und $P(B)>0$ ist, dann ist die bedingte Wahrscheinlichkeit von "$A$ bedingt auf $B$" (oder $A$ unter der Bedingung $B$) definiert durch:

$P(A | B)= \frac{P(A \cap B)}{P(B)}= \frac{P(B|A)\cdot P(A)}{P(B)}$

$P(A \cap B)$ ist die Wahrscheinlichkeit, dass die Ereignisse $A$ und $B$ gemeinsam auftreten.

**Beispiel**

Wenn das Wort "Dear" in acht Ham-E-Mails und in zwei Spam-E-Mails auftaucht, dann ist die bedingte Wahrscheinlichkeit für "Ham bedingt auf Dear" definiert durch:

$p({\rm Ham|Dear}) = \frac{8}{8+2} = \frac{4}{5}$

</div>

<div class="alert alert-block alert-info">
    
<strong>Naive-Bayes-Klassifikation $P(A_1|K_i) \cdot P(A_2|K_i) \cdot \ldots \cdot P(A_n|K_i) \cdot P(K_i)$</strong>

Die Naive-Bayes-Klassifikation ist eine Methode, um einen Wert zu berechnen, der proportional zu einer bedingten Wahrscheinlichkeit von Typ $P(K_i | A_1 \cap A_2 \cap \ldots \cap A_n)$ ist, wobei $K_i$ für die *i*-te Kategorie (z.B. $K_1={\rm Spam}$ und $K_2={\rm Ham}$ bei E-Mails) und $A_1$, $A_2$, $\ldots$, $A_n$ für unterschiedliche Ereignisse stehen (z.B. Wörter in einer E-Mail: $A_1={\rm Hello}$, $A_2={\rm Dear}$, usw.). Gemäss des Satzes von Bayes lässt sich diese Wahrscheinlichkeit folgendermassen berechnen:

$$
P(K_i | A_1 \cap A_2 \cap \ldots \cap A_n) 
= \frac{P(A_1 \cap A_2 \cap \ldots \cap A_n | K_i)\cdot P(K_i)}{P(A_1 \cap A_2 \cap \ldots \cap A_n)}
$$

Unter der (naiven) Annahme, dass die Ereignisse $A_1$, $A_2$, usw. unabhängig voneinander sind, lässt sich $P(A_1 \cap A_2 \cap \ldots \cap A_n | K_i)$ zu einem Produkt der bedingten Wahrscheinlichkeiten $P(A_1 | K_i)$, $P(A_2 | K_i)$, $\ldots$, $P(A_n | K_i)$ vereinfachen:

$$
P(K_i | A_1 \cap A_2 \cap \ldots \cap A_n) 
= \frac{P(A_1|K_i) \cdot P(A_2|K_i) \cdot \ldots \cdot P(A_n|K_i) \cdot P(K_i)}{P(A_1 \cap A_2 \cap \ldots \cap A_n)}
$$

In der Praxis muss beim Einsatz der Naive-Bayes-Klassifikation der Nenner $P(A_1 \cap A_2 \cap \ldots \cap A_n)$
nicht explizit berechnet werden, wenn verglichen wird, welche Kategorie $K_i$ die Wahrscheinlichkeit $P(K_i | A_1 \cap A_2 \cap \ldots \cap A_n)$ maximiert, da er für alle Kategorien gleich ist. Daher vergleichen wir nur die Zähler für die verschiedenen Kategorien $K_i$:

$$
P(A_1|K_i) \cdot P(A_2|K_i) \cdot \ldots \cdot P(A_n|K_i) \cdot P(K_i)
$$

Die Kategorie mit dem grössten Wert bestimmt die Klassifikationsentscheidung.

**Beispiel**

Wenn eine E-Mail den Text "Dear Friend" enhält, dann lässt sich die Wahrscheinlichkeit der Kategorien Ham und Spam mit Hilfe der Formeln 

$P({\rm Dear | Ham}) \cdot P({\rm Friend | Ham}) \cdot P({\rm Ham})$

und 

$P({\rm Dear | Spam}) \cdot P({\rm Friend | Spam}) \cdot P({\rm Spam})$

quantifizieren. Die Kategorie mit dem grössten Produkt wird für die Vorhersage gewählt.

</div>

**Aufgabe 1: A-priori-Wahrscheinlichkeiten für Genres berechnen**

Wir verwenden folgende Häufigkeiten für die Songs in unserer (fiktiven) Musiksammlung:

In [1]:
import pandas as pd

# Series mit den Häufigkeit der Songs
genres = pd.Series({
  "pop": 8404,
  "soul": 387,
  "country": 3869,
  "metal": 1251,
  "hiphop": 92
})
genres

pop        8404
soul        387
country    3869
metal      1251
hiphop       92
dtype: int64

Angenommen Sie ziehen einen zufälligen Song aus der Sammlung. Berechnen Sie die A-priori-Wahrscheinlichkeiten für die verschiedenen Genres als Series
$$
{\rm p\_genres} = 
\begin{bmatrix}
P({\rm pop})\\
P({\rm soul})\\
P({\rm country})\\
P({\rm metal})\\
P({\rm hiphop})\\
\end{bmatrix}
$$
* **Tipp 1**: Mit `n = genres.sum()` berechnen Sie die Gesamtzahl von Songs in Ihrer Sammlung.
* **Tipp 2**: Wenn Sie eine Series-Instanz durch eine Zahl dividieren, werden alle Zellen in der Series durch diese Zahl dividiert.

<img src="https://balloon.tam.ch/share/4ec64dbe79d78eb10116b1a8dfb36958" alt="Series durch Zahl dividieren" width="500">

In [2]:
# Code hier eingeben
n = genres.sum()

p = genres / n

p.sum() # === 1.0, weil die Summe aller Wahrscheinlichkeiten 1 ergibt

p

pop        0.600157
soul       0.027637
country    0.276298
metal      0.089338
hiphop     0.006570
dtype: float64

**Aufgabe 2: Bedingte Wahrscheinlichkeiten berechnen**

Als nächstes importieren wir mit Hilfe von pandas die (fiktive) Häufigkeitsanalysen für die verschiedenen Wörter in unseren Songs. Sie sind bereits aufgeschlüsselt nach Genre. Wir ergänzen sie noch mit Spalten- und Zeilen-Labels.

In [3]:
# Zeilen- und Spalten-Labels als Listen erfassen
rows = ["love","oh","down","got","never","feel","let"]
columns = ["pop", "soul", "country", "metal", "hiphop"]

data = pd.read_csv("./datasets/genres.csv")
lyrics = pd.DataFrame(data.values, index=rows, columns=columns)
lyrics

,pop,soul,country,metal,hiphop
love,12,21,15,4,4
oh,8,10,5,0,2
down,5,4,6,5,5
got,5,7,6,0,9
never,5,5,5,6,3
feel,5,6,4,5,2
let,5,7,4,3,4


**a)** In Metal-Songs in Iher Sammlung kommen offensichtlich die Wörter "oh" und "got" nicht vor. Erklären Sie, weshalb das bei der Naive-Bayes-Klassifikation ein Problem ist und wie man es umgehen kann.

**Antwort:** *Man würde irgendwann durch 0 teilen, was nicht geht.*

**b)** Addieren Sie 1 zu allen Zellen hinzu.

**Tipp**: Wenn man eine Zahl zu einer DataFrame-Instanz addiert, wird sie in allen Zellen hinzuaddiert. Das Resultat muss einer Variablen zugewiesen werden.

In [4]:
# Code hier eingeben
lyrics += 1
lyrics

,pop,soul,country,metal,hiphop
love,13,22,16,5,5
oh,9,11,6,1,3
down,6,5,7,6,6
got,6,8,7,1,10
never,6,6,6,7,4
feel,6,7,5,6,3
let,6,8,5,4,5


**c)** Berechnen Sie die bedingten Wahrscheinlichkeiten für $P({\rm Wort}|{\rm Genre})$ als DataFrame
$$
{\rm cond\_freq} = 
\begin{bmatrix}
P({\rm love|pop}) & P({\rm love|soul}) & P({\rm love|country}) & P({\rm love|metal}) & P({\rm love|hiphop})\\
P({\rm oh|pop})   & P({\rm oh|soul})   & P({\rm oh|country})   & P({\rm oh|metal})   & P({\rm oh|hiphop})  \\
P({\rm down|pop}) & P({\rm down|soul}) & P({\rm down|country}) & P({\rm down|metal}) & P({\rm down|hiphop})\\
P({\rm got|pop})  & P({\rm got|soul})  & P({\rm got|country})  & P({\rm got|metal})  & P({\rm got|hiphop}) \\
P({\rm never|pop})& P({\rm never|soul})& P({\rm never|country})& P({\rm never|metal})& P({\rm never|hiphop})\\
P({\rm feel|pop}) & P({\rm feel|soul}) & P({\rm feel|country}) & P({\rm feel|metal}) & P({\rm feel|hiphop})\\
P({\rm let|pop})  & P({\rm let|soul})  & P({\rm let|country})  & P({\rm let|metal})  & P({\rm let|hiphop}) \\
\end{bmatrix}
$$
* **Tipp 1**: Verwenden Sie `word_count_per_genre = lyrics.sum(axis=0)`, um die Werte im DataFrame `lyrics` spaltenweise zu einer Series zusammenzuaddieren.
* **Tipp 2**: Wenn Sie ein DataFrame durch ein Series dividieren, werden alle DataFrame-Zellen mit der Series-Zelle mit demselben (Spalten-)Label dividiert.

<img src="https://balloon.tam.ch/share/c66a3ebecfa454666985aa1649994399" alt="Division mit Series" width="800">

In [5]:
# Code hier eingeben
word_count_per_genre = lyrics.sum(axis=0)

condition_frequency = lyrics / word_count_per_genre
condition_frequency

,pop,soul,country,metal,hiphop
love,0.250000,0.328358,0.307692,0.166667,0.138889
oh,0.173077,0.164179,0.115385,0.033333,0.083333
down,0.115385,0.074627,0.134615,0.200000,0.166667
got,0.115385,0.119403,0.134615,0.033333,0.277778
never,0.115385,0.089552,0.115385,0.233333,0.111111
feel,0.115385,0.104478,0.096154,0.200000,0.083333
let,0.115385,0.119403,0.096154,0.133333,0.138889


**Aufgabe 3: Songs klassifizieren mit Naive Bayes**

Sagen Sie mit Hilfe der Naive-Bayes-Klassifikation das Genre für des folgenden zwei Songs voraus:

**a)** Song 1: "oh never let love down"

**Antwort**: *Das ist wohl ein Popsong, weil das Produkt der Wahrscheinlichkeiten am grössten ist.*

**b)** Song 2: "got love love love"

**Antwort**: *Dieser Song ist am ehesten ein Soul Song, könnte aber knapp auch ein Country Song sein.*

* **Tipp 1**: Mit `cond_freq.loc['Zeilen-Label']` können Sie im Dataframe `cond_freq` eine ganze Zeile als Series extrahieren (also z.B. mit `cond_freq.loc['oh']` die ganze Zeile mit den bedingten Wahrscheinlichkeiten für das Wort "oh" in allen Genres).
* **Tipp 2**: Wenn Sie eine Series mit einer anderen Series multiplizieren, werden die Werte zellenweise multipliziert. Die Ausrichtung der Series spielt keine Rolle.

<img src="https://balloon.tam.ch/share/f009ae7240abc380a674e157e9e27c98" alt="Zwei Series multiplizieren" width="500">

In [6]:
# a)
song1 = condition_frequency.loc["oh"] * condition_frequency.loc["never"] * condition_frequency.loc["let"] * condition_frequency.loc["love"] * condition_frequency.loc["down"]
song1

pop        0.000066
soul       0.000043
country    0.000053
metal      0.000035
hiphop     0.000030
dtype: float64

In [7]:
# b)
song2 = condition_frequency.loc["got"] * condition_frequency.loc["love"] * condition_frequency.loc["love"] * condition_frequency.loc["love"]

# ODER

song2 = condition_frequency.loc["got"] * (condition_frequency.loc["love"] ** 3)
song2

pop        0.001803
soul       0.004227
country    0.003921
metal      0.000154
hiphop     0.000744
dtype: float64